## Question 1


In [165]:
import pandas as pd
import requests
import urllib.request
from bs4 import BeautifulSoup
import numpy as np
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.10.0 --yes 
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.10.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.10.0              |             py_0          59 KB  conda-forge

The following packages will be UPDATED:

    folium: 0.5.0-py_0 conda-forge --> 0.10.0-py_0 conda-forge


folium-0.10.0        | 59 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


Create Beautiful Soup Object from Wikipedia site

In [147]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, 'html.parser')

Get table from page and iterate over each row, splitting at the '\n' and adding as list to list

In [148]:
table = soup.find_all('table')[0]
list = []
rows = table.find("tbody").find_all("tr")
for row in rows:
    list.append(row.get_text().split('\n')[1:4])


Create Dataframe using first row as columns

In [132]:
df = pd.DataFrame(list[1:], columns=list[0])
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Get the index of rows where borough is not assigned and drop

In [133]:
na_index =df[df['Borough']== 'Not assigned'].index
na_index
df.drop(na_index, inplace=True)

df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


Reset the Index

In [134]:
df.reset_index(drop=True,inplace=True)
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


Group by Postcode, joining neighborhood values with a comma and resetting the index

In [135]:
df = df.groupby(['Borough','Postcode'])['Neighbourhood'].apply(', '.join).reset_index()

In [136]:
df.head()

,Borough,Postcode,Neighbourhood
0,Central Toronto,M4N,Lawrence Park
1,Central Toronto,M4P,Davisville North
2,Central Toronto,M4R,North Toronto West
3,Central Toronto,M4S,Davisville
4,Central Toronto,M4T,"Moore Park, Summerhill East"


Reordering the dataframe according to specs

In [137]:
df = df[['Postcode', 'Borough', 'Neighbourhood']]
df.head()

,Postcode,Borough,Neighbourhood
0,M4N,Central Toronto,Lawrence Park
1,M4P,Central Toronto,Davisville North
2,M4R,Central Toronto,North Toronto West
3,M4S,Central Toronto,Davisville
4,M4T,Central Toronto,"Moore Park, Summerhill East"


Finding the index where neighborhood is not assigned then iterating through index to replace with value for borough

In [149]:
na_index1 =df[df['Neighbourhood']== 'Not assigned'].index

for x in range(len(na_index1)):
    if df.loc[na_index1[x], 'Neighbourhood'] == 'Not assigned' :
        df.loc[na_index1[x], 'Neighbourhood'] = df.loc[na_index1[x],'Borough']


In [145]:
df.shape

(103, 3)

# Question 2

<i>The geocoder package wasn't working well for me, so decided to use the csv file</i> 

In [150]:
!wget -q -O 'Geospatial_Coordinates.csv' http://cocl.us/Geospatial_data

In [151]:
df_geo = pd.read_csv('Geospatial_Coordinates.csv')

In [152]:
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [158]:
df_geo.rename(columns={"Postal Code" : "Postcode"}, inplace=True)
df_merged = pd.merge(df, df_geo, on='Postcode')
df_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
1,M4P,Central Toronto,Davisville North,43.712751,-79.390197
2,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
3,M4S,Central Toronto,Davisville,43.704324,-79.388790
4,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160


# Question 3

In [185]:
df_merged['Borough'].value_counts()

North York          24
Downtown Toronto    18
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East Toronto         5
East York            5
York                 5
Queen's Park         1
Mississauga          1
Name: Borough, dtype: int64

In [162]:
toronto_lat = 43.6532
toronto_long = -79.3832

In [187]:
map_toronto = folium.Map(location=[toronto_lat, toronto_long], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['Borough'], df_merged['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
map_toronto
